Basic Setup - NO need to do on DataProc. But required on fresh notebook
---------------------------------------------------------------------------------------------------

In [1]:
# !apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

E: Package 'openjdk-8-jdk-headless' has no installation candidate


In [2]:
!java -version

openjdk version "1.8.0_322"
OpenJDK Runtime Environment (Temurin)(build 1.8.0_322-b06)
OpenJDK 64-Bit Server VM (Temurin)(build 25.322-b06, mixed mode)


In [3]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [4]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz
!pip install -q findspark

--------------------------------------------------------------------------------------------------
Basic PySpark Code below 
------------------------------------------------

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
import findspark
findspark.init()

Exception: Unable to find py4j in /content/spark-3.0.0-bin-hadoop2.7/python, your SPARK_HOME may not be configured correctly

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.master("local[*]").getOrCreate()
sqlContext = SQLContext(spark)

FileNotFoundError: [Errno 2] No such file or directory: '/content/spark-3.0.0-bin-hadoop2.7/./bin/spark-submit'

In [ ]:
df = sqlContext.createDataFrame([('2020', "Jan", 2000), ('2019', "Jan", 3000), ('2018', "Jan", 4000)],("year", "month", "sales"))
df.show()

------------------------------------------------------------------
No need to do on dataproc, but other notebook, yes. 
Basic Setup to connect to gcs (Google Cloud Storage)
------------------------------------------------------------------------------------

In [9]:
from google.colab import auth
auth.authenticate_user()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  57873      0 --:--:-- --:--:-- --:--:-- 59000
OK
3 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following additional packages will be installed:
  fuse
The following NEW packages will be installed:
  fuse gcsfuse
0 upgraded, 2 newly installed, 0 to remove and 3 not upgraded.
Need to get 12.1 MB of archives.
After this operation, 28.7 MB of additional disk space will be used.
Do you want to continue? [Y/n] 

Installed gcfs to use google cloud storage
----------------------------------------------------------------------------------

Now We copy file from gcs to either local directory or to hadoop cluster
--------------------------------------------------------------------------------------------------------

In [1]:
!gsutil cp gs://nft_csv_ingestion/nft_sales.csv /tmp/nft_sales.csv

AccessDeniedException: 403 486650843789-compute@developer.gserviceaccount.com does not have storage.objects.list access to the Google Cloud Storage bucket.


In [ ]:
!gsutil cp /tmp/nft_sales.csv gs://nft_csv_ingestion/nft_sales.csv 

In [ ]:
!hadoop dfs -cp  gs://nft_csv_ingestion/nft_sales.csv /tmp/nft_sales.csv

In [ ]:
df = sqlContext.read.format("csv").option("header","true").option("delimiter", ",").load("/tmp/nft_sales.csv")
df.show(5)


In [ ]:
df = sqlContext.read.format("csv").option("header","true").option("delimiter", ",").load("file:///tmp/nft_sales.csv")
df.show(5)



---------------------------------------------

Sql lite db etl analysis
-------------------------------------


In [18]:
!hadoop dfs -cp  gs://nft_sql_lite_db/nfts.sqlite /tmp/nfts.sqlite


cp: `/tmp/nfts.sqlite': File exists


In [5]:
!gsutil cp gs://nft_sql_lite_db/nfts.sqlite /tmp/nfts.sqlite

Copying gs://nft_sql_lite_db/nfts.sqlite...
/ [1 files][  6.9 GiB/  6.9 GiB]   55.0 MiB/s                                   
Operation completed over 1 objects/6.9 GiB.                                      


In [6]:
!ls /tmp/nfts.sqlite

/tmp/nfts.sqlite


In [51]:
!hadoop fs -ls /tmp

Found 3 items
drwxrwxrwt   - hdfs hadoop          0 2022-04-07 06:21 /tmp/hadoop-yarn
drwx-wx-wx   - hive hadoop          0 2022-04-07 06:21 /tmp/hive
-rw-r--r--   1 root hadoop 7433949184 2022-04-10 16:43 /tmp/nfts.sqlite


In [ ]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

sqlContext.read.format("jdbc").options(url ="jdbc:sqlite:/tmp/nfts.sqlite", driver="org.sqlite.JDBC", dbtable="current_owners").load().take(10) 

In [ ]:
myrdd2 = spark.read.format("jdbc").option("url", "jdbc:sqlite:/tmp/nfts.sqlite").option("dbtable", "current_owners").option("driver", "org.sqlite.JDBC").load()

In [ ]:
!cd /tmp
!curl https://repo1.maven.org/maven2/org/xerial/sqlite-jdbc/3.28.0/sqlite-jdbc-3.28.0.jar

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:///tmp/sqlite-jdbc-3.36.0.3.jar pyspark-shell'

In [ ]:
driver = "org.sqlite.JDBC"
path = "/tmp/nfts.sqlite"
url = "jdbc:sqlite:" + path
tablename = "current_owners"

In [ ]:
df_albums = sqlContext.read.format("jdbc").option("url", url).option("dbtable", tablename).option("driver", driver).load()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark3 =SparkSession.builder \
  .master('local[*]') \
  .appName('conversions') \
  .config('spark.jars.packages', 'org.xerial:sqlite-jdbc:3.34.0') \
  .getOrCreate() 
sqlContext = SQLContext(spark3)

In [5]:
!pyspark --packages org.xerial:sqlite-jdbc:3.34.0
!exit

Python 3.8.12 | packaged by conda-forge | (default, Jan 30 2022, 23:42:07) 
[GCC 9.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.xerial#sqlite-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-deda2f06-212d-4c22-9a00-65e42e35fc3b;1.0
	confs: [default]
	found org.xerial#sqlite-jdbc;3.34.0 in central
:: resolution report :: resolve 161ms :: artifacts dl 3ms
	:: modules in use:
	org.xerial#sqlite-jdbc;3.34.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-------------------------------------------------

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark4 =SparkSession.builder \
  .master('local[*]') \
  .appName('conversions') \
  .config('spark.jars', '/content/sqlite-jdbc-3.8.6.jar') \
  .getOrCreate() 
sqlContext = SQLContext(spark4)

22/04/10 16:54:36 WARN org.apache.spark.deploy.DependencyUtils: Local jar /content/sqlite-jdbc-3.8.6.jar does not exist, skipping.
22/04/10 16:54:37 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/10 16:54:37 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/10 16:54:37 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/10 16:54:37 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
22/04/10 16:54:37 INFO org.sparkproject.jetty.util.log: Logging initialized @3413ms to org.sparkproject.jetty.util.log.Slf4jLog
22/04/10 16:54:37 INFO org.sparkproject.jetty.server.Server: jetty-9.4.40.v20210413; built: 2021-04-13T20:42:42.668Z; git: b881a572662e1943a14ae12e7e1207989f218b74; jvm 1.8.0_322-b06
22/04/10 16:54:37 INFO org.sparkproject.jetty.server.Server: Started @3531ms
22/04/10 16:54:37 INFO org.sparkproject.jetty.server.AbstractConnector: Started ServerConnector@4c4776d7{HTTP/1.1, (http/1.1)}{0.0.0.0:33517}
22/

In [2]:
spark4.conf.get("spark.jars")

'/content/sqlite-jdbc-3.8.6.jar'

In [6]:
df = spark4.read.format('jdbc') \
        .options(driver='org.sqlite.JDBC', dbtable='current_owners',
                 url='jdbc:sqlite:gs:///nfts.sqlite')\
        .load()

Py4JJavaError: An error occurred while calling o81.load.
: java.lang.ClassNotFoundException: org.sqlite.JDBC
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:102)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:102)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:102)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:38)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:32)
	at org.apache.spark.sql.executio

In [16]:
import sqlite3

import pandas as pd

db_path = '/tmp/nfts.sqlite'

query = 'SELECT * from current_owners'

conn = sqlite3.connect(db_path)

a_pandas_df = pd.read_sql_query(query, conn)

a_spark_df = sqlContext.createDataFrame(data=a_pandas_df)

In [18]:
a_spark_df.show(5)

22/04/11 12:31:47 WARN org.apache.spark.scheduler.TaskSetManager: Stage 0 contains a task of very large size (173673 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------+--------------------+
|         nft_address|token_id|               owner|
+--------------------+--------+--------------------+
|0x00000000000b7F8...|       0|0xb776cAb26B9e6Be...|
|0x00000000000b7F8...|       1|0x8A73024B39A4477...|
|0x00000000000b7F8...|      10|0x5e5C817E9264B46...|
|0x00000000000b7F8...|      11|0x8376f63c13b99D3...|
|0x00000000000b7F8...|      12|0xb5e34552F32BA92...|
+--------------------+--------+--------------------+
only showing top 5 rows



Basic pyspark working in Google Colab

In [8]:
!pip install nfts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.6/499.6 KB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 61.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 KB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 KB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 KB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 87.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


  Created wheel for lru-dict: filename=lru_dict-1.1.7-cp38-cp38-linux_x86_64.whl size=32617 sha256=f971401cfadac1a0ff8b8ef60c494e7cd2a48d60edded857916cbb61d98b5013
  Stored in directory: /root/.cache/pip/wheels/ab/ed/82/d47759a6e2416fb78d6ee81c02a970a50d5dc6fc270b5a3242
  Created wheel for bitarray: filename=bitarray-1.2.2-cp38-cp38-linux_x86_64.whl size=167564 sha256=773f0ff3ef2fed2b5fabbb58f807e60de45847d61228fdaef706484ec062dae2
  Stored in directory: /root/.cache/pip/wheels/41/36/95/5b4eca059535a8400e8f4ca38f4883ea1801bb221fbd8170df
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42723 sha256=09028b2570446eede55379841fa445142b1b612cbbd63d810e654a410021719e
  Stored in directory: /root/.cache/pip/wheels/d8/af/19/fb896f509a437aca2dcf62583e84d7fb2cd5b628c1564a609c
  Created wheel for varint: filename=varint-1.0.2-py3-none-any.whl size=1980 sha256=d87fcf4a5be0486045be006900720dd7d1fe32bc43c2c1063ecd14a8d4cdeafa
  Stored in directory: /root/.cache/pip

In [7]:
import os
import sqlite3

import matplotlib.pyplot as plt
import nfts.dataset
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.special import zeta

In [8]:
!ls /tmp


2928c921-5892-4800-9675-c11030c550b1_resources
523ded43-4f7d-4599-b98a-69e58ea31661_resources
SERVERENGINE_SOCKETMANAGER_2022-04-11T12:15:32Z_1900
cluster
dataproc-agent3526609400769203650
fdf96458-ddc5-453b-96e1-5f9ad9292cf0_resources
hadoop-unjar22182705041423930
hadoop-unjar8040116867953293139
hadoop-yarn-yarn
hive
hsperfdata_hdfs
hsperfdata_hive
hsperfdata_knox
hsperfdata_mapred
hsperfdata_root
hsperfdata_spark
hsperfdata_yarn
jetty-0_0_0_0-10002-hive-service-3_1_2_jar-_-any-9158813490248169229
jetty-0_0_0_0-8042-hadoop-yarn-common-3_2_2_jar-_-any-519076419844954091
jetty-0_0_0_0-8088-hadoop-yarn-common-3_2_2_jar-_-any-4291174230845628428
jetty-0_0_0_0-8188-hadoop-yarn-common-3_2_2_jar-_-any-5140851581501485200
jetty-0_0_0_0-8188-tez-ui-0_9_2_war-_tez-ui-any-7482519492520096315
jetty-0_0_0_0-9868-secondary-_-any-4631692437528764687
jetty-0_0_0_0-9870-hdfs-_-any-3585418971285134640
jetty-localhost-38321-datanode-_-any-3277174947876911983
jetty-nft-cluster-m-19888-hadoop-yarn-common-

In [9]:
DATASET_PATH = "/tmp/nfts.sqlite"
# DATASET_PATH = "gs://nft_sql_lite_db/nfts.sqlite"

In [10]:
ds = nfts.dataset.FromSQLite(DATASET_PATH)

In [11]:
current_owners_df = ds.load_dataframe("current_owners")

In [6]:
current_owners_df

NameError: name 'current_owners_df' is not defined